In [1]:
import pandas as pd

## Load Data

In [2]:
january = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')

In [3]:
february = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

## Q1 - Columns

In [4]:
len(january.columns)

19

## Q2 - Compute duration

In [5]:
january.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


Take the difference between dropoff and pickup

In [6]:
january['duration'] = (january['tpep_dropoff_datetime'] - january['tpep_pickup_datetime']).dt.total_seconds()/60

In [7]:
january['duration'].std()

42.594351241920904

## Q3 - Dropping outliers

Get original length

In [8]:
orig_length = len(january['duration'])

In [9]:
january = january[(january['duration'] >= 1) & (january['duration'] <= 60)]

In [10]:
len(january)/orig_length

0.9812202822125979

## Q4 - One-hot encoding

In [11]:
from sklearn.feature_extraction import DictVectorizer

cat_cols = ['PULocationID', 'DOLocationID']
df_cat = january[cat_cols].astype(str)

records = df_cat.to_dict(orient='records')

dv = DictVectorizer(sparse=True)
X = dv.fit_transform(records) 

feature_names = dv.get_feature_names_out()

print(X.shape) 

(3009173, 515)


## Q5 - Training a model

In [12]:
from sklearn.linear_model import LinearRegression

In [13]:
import numpy as np

In [14]:
y = january['duration']

In [15]:
reg = LinearRegression().fit(X, y)

In [16]:
y_hat = reg.predict(X)

In [17]:
np.sqrt(np.mean((y_hat - y)**2))

7.649261027919879

## Q6 - Evaluating a model

In [18]:
february['duration'] = (february['tpep_dropoff_datetime'] - february['tpep_pickup_datetime']).dt.total_seconds()/60

In [19]:
february = february[(february['duration'] >= 1) & (february['duration'] <= 60)]

In [20]:
X_test = dv.transform(february[cat_cols].astype(str).to_dict(orient='records'))
y_pred_test = reg.predict(X_test)

In [21]:
y_hat_feb = reg.predict(X_test)

In [22]:
np.sqrt(np.mean((y_hat_feb - february['duration'])**2))

7.8118326382731755